# 1. 라이브러리 임포트

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
# 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tqdm
from transformers import Trainer, TrainingArguments
import re
import pyarrow as pa
from datasets import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.model_selection import train_test_split
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')

# 2. 데이터불러오기 및 EDA

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/아이펠톤/DKTC data/train.csv')

In [ ]:
data

# 3. 데이터 전처리및 타입변경

In [ ]:
data.rename(columns={'class':'labels'}, inplace=True)

# change labels to int
label_list = {'기타 괴롭힘 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '협박 대화':3}
data['labels'] = data['labels'].apply(lambda x: label_list[x])

# convert to Huggingface dataset
import pyarrow as pa
from datasets import Dataset

data = Dataset(pa.Table.from_pandas(data))

# **klue/roberta-base**

In [ ]:
model_path = "klue/roberta-base"

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels = 4)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/752k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classif

In [ ]:
def transform(data):
  return tokenizer(
      data['conversation'],
      truncation = True,
      padding = 'max_length',
      return_token_type_ids = True,
      )

encoded_data = data.map(transform, batched=True)

from sklearn.model_selection import train_test_split
train_dataset, validation_dataset= encoded_data.train_test_split(test_size=0.1).values()

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='klue/roberta-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
data['conversation'][0]

'지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.\n 정말 잘못했습니다.\n 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.\n 죄송합니다. 정말 잘못했습니다.\n 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.\n 선택 못하겠습니다. 한번만 도와주세요.\n 그냥 다 죽여버려야겠군. 이의 없지?\n 제발 도와주세요.'

In [ ]:
tokenizer.decode(encoded_data['input_ids'][0])

'[CLS] 지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게. 정말 잘못했습니다. 너가 선택해. 너가 죽을래 네 가족을 죽여줄까. 죄송합니다. 정말 잘못했습니다. 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야. 선택 못하겠습니다. 한번만 도와주세요. 그냥 다 죽여버려야겠군. 이의 없지? 제발 도와주세요. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [ ]:
encoded_data['input_ids'][0]

In [ ]:
metric = load_metric("f1")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels, average="micro")

In [ ]:
output_dir = '/content/drive/MyDrive/아이펠톤/미니아펠톤/model_data'

training_arguments = TrainingArguments(
    output_dir, 
    evaluation_strategy="epoch", #evaluation하는 빈도
    learning_rate = 2e-5, 
    per_device_train_batch_size = 16, 
    per_device_eval_batch_size = 16, 
    num_train_epochs = 3,
    weight_decay = 0.01
)

In [ ]:
trainer = Trainer(
    model=model,                           
    args=training_arguments,                  
    train_dataset=train_dataset,    
    eval_dataset=validation_dataset,      
    compute_metrics=compute_metrics
)
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: conversation, idx. If conversation, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3555
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 669
  Number of trainable parameters = 110621188


Epoch,Training Loss,Validation Loss,F1
1,No log,0.252381,0.921519
2,No log,0.189114,0.926582
3,0.379700,0.210209,0.941772


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: conversation, idx. If conversation, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: conversation, idx. If conversation, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 395
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/아이펠톤/미니아펠톤/model_data/checkpoint-500
Configuration saved in /content/drive/MyDrive/아이펠톤/미니아펠톤/model_data/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/아이펠톤/미니아펠톤/model_data/checkpoint-500/pytorch_model.b

TrainOutput(global_step=669, training_loss=0.3163399147524071, metrics={'train_runtime': 974.125, 'train_samples_per_second': 10.948, 'train_steps_per_second': 0.687, 'total_flos': 2806129794723840.0, 'train_loss': 0.3163399147524071, 'epoch': 3.0})